<a href="https://colab.research.google.com/github/frisk0zisan/nlp100/blob/main/Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8章：ニューラルネット
第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ。  
->今回はPytorchを使う。

## 70. 単語ベクトルの和による特徴量Permalink

問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例$x_i$の特徴ベクトル$\boldsymbol{x}_i$を並べた行列$X$と正解ラベルを並べた行列（ベクトル）$Y$を作成したい．

$$
X = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \dots \\ 
  \boldsymbol{x}_n \\ 
\end{pmatrix} \in \mathbb{R}^{n \times d},
Y = \begin{pmatrix} 
  y_1 \\ 
  y_2 \\ 
  \dots \\ 
  y_n \\ 
\end{pmatrix} \in \mathbb{N}^{n}
$$


 ここで，$n$は学習データの事例数であり，$\boldsymbol x_i \in \mathbb{R}^d$と$y_i \in \mathbb N$はそれぞれ，$i \in \{1, \dots, n\}$番目の事例の特徴量ベクトルと正解ラベルを表す．
 なお，今回は「ビジネス」「科学技術」「エンターテイメント」「健康」の4カテゴリ分類である．$\mathbb N_{<4}$で$4$未満の自然数（$0$を含む）を表すことにすれば，任意の事例の正解ラベル$y_i$は$y_i \in \mathbb N_{<4}$で表現できる．
 以降では，ラベルの種類数を$L$で表す（今回の分類タスクでは$L=4$である）．

 $i$番目の事例の特徴ベクトル$\boldsymbol x_i$は，次式で求める．

 $$\boldsymbol x_i = \frac{1}{T_i} \sum_{t=1}^{T_i} \mathrm{emb}(w_{i,t})$$

 ここで，$i$番目の事例は$T_i$個の（記事見出しの）単語列$(w_{i,1}, w_{i,2}, \dots, w_{i,T_i})$から構成され，$\mathrm{emb}(w) \in \mathbb{R}^d$は単語$w$に対応する単語ベクトル（次元数は$d$）である．  
 すなわち，$i$番目の事例の記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したものが$\boldsymbol x_i$である．今回は単語ベクトルとして，問題60でダウンロードしたものを用いればよい．$300$次元の単語ベクトルを用いたので，$d=300$である．  
 $i$番目の事例のラベル$y_i$は，次のように定義する．

$$
y_i = \begin{cases}
0 & (\mbox{記事}\boldsymbol x_i\mbox{が「ビジネス」カテゴリの場合}) \\
1 & (\mbox{記事}\boldsymbol x_i\mbox{が「科学技術」カテゴリの場合}) \\
2 & (\mbox{記事}\boldsymbol x_i\mbox{が「エンターテイメント」カテゴリの場合}) \\
3 & (\mbox{記事}\boldsymbol x_i\mbox{が「健康」カテゴリの場合}) \\
\end{cases}
$$

なお，カテゴリ名とラベルの番号が一対一で対応付いていれば，上式の通りの対応付けでなくてもよい．

以上の仕様に基づき，以下の行列・ベクトルを作成し，ファイルに保存せよ．

 + 学習データの特徴量行列: $X_{\rm train} \in \mathbb{R}^{N_t \times d}$
 + 学習データのラベルベクトル: $Y_{\rm train} \in \mathbb{N}^{N_t}$
 + 検証データの特徴量行列: $X_{\rm valid} \in \mathbb{R}^{N_v \times d}$
 + 検証データのラベルベクトル: $Y_{\rm valid} \in \mathbb{N}^{N_v}$
 + 評価データの特徴量行列: $X_{\rm test} \in \mathbb{R}^{N_e \times d}$
 + 評価データのラベルベクトル: $Y_{\rm test} \in \mathbb{N}^{N_e}$

なお，$N_t, N_v, N_e$はそれぞれ，学習データの事例数，検証データの事例数，評価データの事例数である．

6章の問50と同様の処理を行う。  
データダウンロード後に、データ分割後に保存する。

In [1]:
## データのダウンロード
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
!unzip NewsAggregatorDataset.zip

--2021-01-31 02:34:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  24.9MB/s    in 1.1s    

2021-01-31 02:34:38 (24.9 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [2]:
!unzip "NewsAggregatorDataset.zip"

Archive:  NewsAggregatorDataset.zip
replace 2pageSessions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/._2pageSessions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace newsCorpora.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/._newsCorpora.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/._readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [3]:
# 読込時のエラー回避のためダブルクォーテーションをシングルクォーテーションに置換
!sed -e 's/"/'\''/g' ./newsCorpora.csv > ./newsCorpora_re.csv

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

## データ読み込み
df = pd.read_csv('./newsCorpora_re.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

## データ抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']),['TITLE', 'CATEGORY']]

## データ分割
train, valid_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123, stratify=df['CATEGORY'])
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, random_state=123, stratify=valid_test['CATEGORY'])

## データ保存
train.to_csv('./train.txt', sep='\t', index=False)
valid.to_csv('./valid.txt', sep='\t', index=False)
test.to_csv('./test.txt', sep='\t', index=False )

In [5]:
## 事例数の確認
print("カテゴリ:(b = business, t = science and technology, e = entertainment, m = health)")
print('---学習データ---')
print(train['CATEGORY'].value_counts())
print('---検証データ---')
print(valid['CATEGORY'].value_counts())
print('---評価データ---')
print(test['CATEGORY'].value_counts())

カテゴリ:(b = business, t = science and technology, e = entertainment, m = health)
---学習データ---
b    4501
e    4235
t    1220
m     728
Name: CATEGORY, dtype: int64
---検証データ---
b    563
e    529
t    153
m     91
Name: CATEGORY, dtype: int64
---評価データ---
b    563
e    530
t    152
m     91
Name: CATEGORY, dtype: int64


In [6]:
df = pd.concat([train, valid, test], axis=0)
df.reset_index(drop=True, inplace=True)

7章の問60と同様の処理を行う。以下7章問60  
>Google Newsデータセット（約1,000億単語）での[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit)（300万単語・フレーズ，300次元）をダウンロードし，”United States”の単語ベクトルを表示せよ．ただし，”United States”は内部的には”United_States”と表現されていることに注意せよ．

学習済み単語ベクトルをダウンロードして、ロードする。

In [7]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'

In [8]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

n
--2021-01-31 02:35:18--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.97.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.97.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  81.8MB/s    in 20s     

2021-01-31 02:35:37 (79.6 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



Genismを用いて単語ベクトルを読み込む。

In [9]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)


読み込んだ後は、ベクトル化したい単語を指定するだけで簡単に単語ベクトルを得ることができる。

In [10]:
model['Apple'][:5]

array([-0.17480469,  0.0300293 , -0.21679688,  0.15625   , -0.35742188],
      dtype=float32)

word2vecを用いることで単語の類義語や単語間や文章間の類義度を計ることができる。  
以下に「Apple」の類義語を示す。

In [11]:
model.most_similar('Apple', topn=5)

[('Apple_AAPL', 0.7456985712051392),
 ('Apple_Nasdaq_AAPL', 0.7300410270690918),
 ('Apple_NASDAQ_AAPL', 0.7175089716911316),
 ('Apple_Computer', 0.7145973443984985),
 ('iPhone', 0.6924266219139099)]

後にGPUで学習するためPytorchを使用する。  
そのため、Tensor型に変換する。
### PythorchのTensor型について
numpyのndarray型ととても似ている。  
Tensor型を使うと嬉しい点
- GPUを利用できる
- 勾配情報を簡単に求められる

以下にTensor型の練習コードを書く



In [12]:
import torch

In [13]:
## GPUを利用できる
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
c = torch.zeros([4], device=device)
print(c)

tensor([0., 0., 0., 0.], device='cuda:0')


In [14]:
## 勾配情報を簡単に求められる
x = torch.tensor(5.0, requires_grad=True)
y = 10*x + 20
y.backward()

print(y)
print(x)
print(x.grad)

tensor(70., grad_fn=<AddBackward0>)
tensor(5., requires_grad=True)
tensor(10.)


特徴ベクトルをTensor型で作成する。
特徴ベクトルの以下の通り。（再掲）

 $i$番目の事例の特徴ベクトル$\boldsymbol x_i$

 $$\boldsymbol x_i = \frac{1}{T_i} \sum_{t=1}^{T_i} \mathrm{emb}(w_{i,t})$$

$T_i$：単語列の長さ  
$(w_{i,1}, w_{i,2}, \dots, w_{i,T_i})$：単語列  
$\mathrm{emb}(w) \in \mathbb{R}^d$：単語ベクトル  
$d$：単語$w$に対応する単語ベクトル（次元数）

In [15]:
import string

def make_w2v(text):
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  words = text.translate(table).split()

  vec = [model[word] for word in words if word in model]

  return torch.tensor((1/len(words))*sum(vec))


In [16]:
## 特徴ベクトル作成
X_train = torch.stack([make_w2v(text) for text in train['TITLE']])
X_valid = torch.stack([make_w2v(text) for text in valid['TITLE']])
X_test = torch.stack([make_w2v(text) for text in test['TITLE']])

In [17]:
print(X_train.size())
print(X_train[0:5])
print(X_valid.size())
print(X_valid[0:5])

torch.Size([10684, 300])
tensor([[ 0.0837,  0.0056,  0.0068,  ...,  0.0751,  0.0433, -0.0868],
        [ 0.0242,  0.0236, -0.0842,  ..., -0.0930, -0.0435, -0.0082],
        [ 0.0577, -0.0159, -0.0780,  ..., -0.0421,  0.1229,  0.0876],
        [-0.0555,  0.0496,  0.0620,  ..., -0.0136,  0.0390, -0.0206],
        [-0.0259,  0.0775, -0.0256,  ..., -0.0364,  0.1126,  0.0063]])
torch.Size([1336, 300])
tensor([[-0.0264,  0.1098,  0.0382,  ...,  0.0056,  0.0513,  0.1587],
        [ 0.0712,  0.1152, -0.0822,  ..., -0.0151,  0.0091,  0.0217],
        [-0.0058,  0.0320, -0.0094,  ...,  0.0195,  0.0591,  0.0504],
        [-0.0187,  0.0179,  0.1041,  ..., -0.0404,  0.1052, -0.0224],
        [ 0.0134,  0.0070,  0.0432,  ...,  0.0462,  0.0857, -0.0158]])


次にラベルベクトルを作成する。  
以下に$i$番目の事例のラベル$y_i$の定義を示す（再掲）

$$
y_i = \begin{cases}
0 & (\mbox{記事}\boldsymbol x_i\mbox{が「ビジネス」カテゴリの場合}) \\
1 & (\mbox{記事}\boldsymbol x_i\mbox{が「科学技術」カテゴリの場合}) \\
2 & (\mbox{記事}\boldsymbol x_i\mbox{が「エンターテイメント」カテゴリの場合}) \\
3 & (\mbox{記事}\boldsymbol x_i\mbox{が「健康」カテゴリの場合}) \\
\end{cases}
$$

In [18]:
label_dic ={'b':0, 't':1, 'e':2, 'm':3}
y_train = torch.tensor([label_dic[label] for label in train['CATEGORY']])
y_valid = torch.tensor([label_dic[label] for label in valid['CATEGORY']])
y_test =  torch.tensor([label_dic[label] for label in test['CATEGORY']])


In [19]:
print(y_train.size())
print(y_train[0:5])
print(y_valid.size())
print(y_valid[0:5])

torch.Size([10684])
tensor([0, 1, 3, 2, 0])
torch.Size([1336])
tensor([1, 2, 0, 0, 0])


In [20]:
## 保存
torch.save(X_train, 'X_train.pt')
torch.save(X_valid, 'X_valid.pt')
torch.save(X_test, 'X_test.pt')
torch.save(y_train, 'y_train.pt')
torch.save(y_valid, 'y_valid.pt')
torch.save(y_test, 'y_test.pt')

## 71. 単層ニューラルネットワークによる予測
問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．

$$ 
\hat{y}_1=softmax(x_1W),\\\hat{Y}=softmax(X_{[1:4]}W)
$$


ただし，$softmax$はソフトマックス関数，$X_{[1:4]}∈\mathbb{R}^{4×d}$は特徴ベクトル$x_1$,$x_2$,$x_3$,$x_4$を縦に並べた行列である．

$$
X_{[1:4]}=\begin{pmatrix}x_1\\x_2\\x_3\\x_4\end{pmatrix}
$$

行列$W \in \mathbb{R}^{d \times L}$は単層ニューラルネットワークの重み行列で，ここではランダムな値で初期化すればよい（問題73以降で学習して求める）．  
なお，$\hat{\boldsymbol y_1} \in \mathbb{R}^L$は未学習の行列$W$で事例$x_1$を分類したときに，各カテゴリに属する確率を表すベクトルである．
同様に，$\hat{Y} \in \mathbb{R}^{n \times L}$は，学習データの事例$x_1, x_2, x_3, x_4$について，各カテゴリに属する確率を行列として表現している．


In [21]:
len(X_train)

10684

In [22]:
import torch.nn as nn

In [23]:
INPUT_FEATURES = 300  # 入力（特徴）の数： 300
OUTPUT_NEURONS = 4  # ニューロンの数： 4

## torch.nn.Moduleクラスを継承する
class Net(nn.Module):
  def __init__(self, input_features, output_neurons):
    super(Net, self).__init__()
    ## 層（layer）を定義
    self.layer = nn.Linear(input_features, output_neurons, bias=False) #Linearは「全結合層」を指す
    nn.init.normal_(self.layer.weight, 0.0, 1.0) # 正規乱数で重み初期化

  ## フォワードパスを定義
  def forward(self, input):
    output = self.layer(input)
    ##　層を重ねる場合は「出力：output」を次の層の「入力：input」に使う。
    return output


In [36]:
model = Net(INPUT_FEATURES, OUTPUT_NEURONS)
print(model)
y_hat_1 = torch.softmax(model(X_train[:1]), dim=-1)
print(y_hat_1)
Y_hat = torch.softmax(model(X_train[:4]), dim=-1)
print(Y_hat)

Net(
  (layer): Linear(in_features=300, out_features=4, bias=False)
)
tensor([[0.0296, 0.1538, 0.2084, 0.6082]], grad_fn=<SoftmaxBackward>)
tensor([[0.0296, 0.1538, 0.2084, 0.6082],
        [0.0360, 0.1979, 0.5756, 0.1905],
        [0.0299, 0.0502, 0.8569, 0.0631],
        [0.1817, 0.0737, 0.1164, 0.6282]], grad_fn=<SoftmaxBackward>)


## 72. 損失と勾配の計算Permalink

学習データの事例x1
と事例集合x1,x2,x3,x4
に対して，クロスエントロピー損失と，行列W
に対する勾配を計算せよ．なお，ある事例xi
に対して損失は次式で計算される．

$$l_i=−log[事例x_iがy_iに分類される確率]$$

ただし，事例集合に対するクロスエントロピー損失は，その集合に含まれる各事例の損失の平均とする．